In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Qaanaaq,GL,77.48,-69.36,43.84,67,100,7.67,overcast clouds,0.00,0
1,1,Muriwai Beach,NZ,-36.82,174.45,57.00,84,17,1.01,light rain,0.14,0
2,2,Vila Franca Do Campo,PT,37.72,-25.43,68.00,77,75,4.70,broken clouds,0.00,0
3,3,Khatanga,RU,71.97,102.50,48.85,90,100,20.83,light rain,0.91,0
4,4,Hermanus,ZA,-34.42,19.23,52.00,79,100,7.00,moderate rain,1.09,0


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rainingWeather = str(input("Do you want it to be raining? (yes/no) "))
snowingWeather = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if rainingWeather == "no" else city_data_df["Rain (inches)"] > 0) & 
                                       (city_data_df["Snow (inches)"] == 0 if snowingWeather == "no" else city_data_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
17,17,Vaini,TO,-21.20,-175.20,77.00,69,20,11.41,few clouds,0.0,0
18,18,Basco,PH,20.45,121.97,84.88,68,17,2.37,few clouds,0.0,0
30,30,Monrovia,LR,6.30,-10.80,77.00,94,75,6.93,broken clouds,0.0,0
33,33,Pathein,MM,16.78,94.73,81.50,83,100,6.31,overcast clouds,0.0,0
35,35,Kushikino,JP,31.72,130.27,78.80,100,75,10.29,light intensity shower rain,0.0,0
39,39,Kazerun,IR,29.62,51.65,82.04,15,0,7.47,clear sky,0.0,0
52,52,Kahului,US,20.89,-156.47,89.60,43,20,27.51,few clouds,0.0,0
56,56,Katherine,AU,-14.47,132.27,82.40,42,40,10.29,scattered clouds,0.0,0
58,58,Dingle,PH,11.00,122.67,85.06,72,99,0.65,overcast clouds,0.0,0
65,65,Hirara,JP,24.80,125.28,87.80,74,40,13.87,scattered clouds,0.0,0


In [7]:
preferred_cities_df.count()

City_ID                138
City                   138
Country                138
Lat                    138
Lng                    138
Max Temp               138
Humidity               138
Cloudiness             138
Wind Speed             138
Current Description    138
Rain (inches)          138
Snow (inches)          138
dtype: int64

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Vaini,TO,77.00,few clouds,-21.20,-175.20,
18,Basco,PH,84.88,few clouds,20.45,121.97,
30,Monrovia,LR,77.00,broken clouds,6.30,-10.80,
33,Pathein,MM,81.50,overcast clouds,16.78,94.73,
35,Kushikino,JP,78.80,light intensity shower rain,31.72,130.27,
39,Kazerun,IR,82.04,clear sky,29.62,51.65,
52,Kahului,US,89.60,few clouds,20.89,-156.47,
56,Katherine,AU,82.40,scattered clouds,-14.47,132.27,
58,Dingle,PH,85.06,overcast clouds,11.00,122.67,
65,Hirara,JP,87.80,scattered clouds,24.80,125.28,


In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Vaini,TO,77.00,few clouds,-21.20,-175.20,Keleti Beach Resort
18,Basco,PH,84.88,few clouds,20.45,121.97,Nathaniel's Lodge
30,Monrovia,LR,77.00,broken clouds,6.30,-10.80,Eco-Hotel Hall & Restaurant
33,Pathein,MM,81.50,overcast clouds,16.78,94.73,Pathein Hotel
35,Kushikino,JP,78.80,light intensity shower rain,31.72,130.27,Tajiri Inn
39,Kazerun,IR,82.04,clear sky,29.62,51.65,آپارتمان بزرگ اویسا
52,Kahului,US,89.60,few clouds,20.89,-156.47,Maui Seaside Hotel
56,Katherine,AU,82.40,scattered clouds,-14.47,132.27,Pine Tree Motel
58,Dingle,PH,85.06,overcast clouds,11.00,122.67,Camp Pasica
65,Hirara,JP,87.80,scattered clouds,24.80,125.28,Hotel Atoll Emerald Miyakojima


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")